## アプリケーション デバッグ

In [1]:
# モジュール
from main import create_app
from models import *


In [2]:
# create_app() SQLAlchemy定義済み
app = create_app()
# app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///..\\..\\data\\user_database.db"


### SQLAlchemy動作確認
- https://flask-sqlalchemy.palletsprojects.com/en/3.1.x/queries/
- https://flask-sqlalchemy.palletsprojects.com/en/3.1.x/legacy-query/

In [3]:
# db start db = SQLAlchemy()はmodule内で定義されているので重複を避ける
# db.init_app(app)
# app.app_context().push()

In [4]:
# db作成
# db.create_all()

In [5]:
# create_appでインスタンス化された際 user_listが作成済み
user_list = UserList()
all_user = user_list._get_all()
print(all_user)
for user in all_user:
    print(user.user_name, user.pass_word)


2024-03-03 00:25:44,013 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-03 00:25:44,018 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users
2024-03-03 00:25:44,020 INFO sqlalchemy.engine.Engine [generated in 0.00242s] ()
[<User #thisistoken>, <User #wowow>, <User #goose>, <User #letseat>, <User #good>]
username user
amazingman amazing
god god
chika eat
goodman goodgood


In [28]:
# add
user_list.add_user(user_token="#letseat", display_name="chika", pass_word="eat") 
# assert db.session.query(User).filter_by(user_token="#thisistoken").first(), "Userオブジェクトが作成されていません"


2024-03-02 20:55:36,275 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:55:36,277 INFO sqlalchemy.engine.Engine INSERT INTO users (user_token, user_name, pass_word, done) VALUES (?, ?, ?, ?)
2024-03-02 20:55:36,278 INFO sqlalchemy.engine.Engine [cached since 24.69s ago] ('#letseat', 'chika', 'eat', 0)
2024-03-02 20:55:36,281 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
# errorが出た際にセッション
db.session.rollback()
user_list.varified = True
db.session.commit()

In [29]:
# check UserList
user = user_list.get_user_info("#letseat")

# 要素の取得
# name 
user_name = user.user_name
# password
user_pass = user.pass_word
# token
user_token = user.user_token

print("\n")
print(f"user_name: {user_name}")
print(f"user_pass: {user_pass}")
print(f"user_token: {user_token}")


2024-03-02 20:55:48,211 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:55:48,214 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users 
WHERE users.user_token = ?
 LIMIT ? OFFSET ?
2024-03-02 20:55:48,215 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('#letseat', 1, 0)


user_name: chika
user_pass: eat
user_token: #letseat


In [44]:
# 404エラー受け取り
try:
    # 例外発生受け取り
    c = User.query.filter_by(user_name="amazingman").first_or_404()
    print(c)
except BaseException as e:
    print("This is 404 Error", e)


2024-03-02 21:03:15,970 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2024-03-02 21:03:15,971 INFO sqlalchemy.engine.Engine [cached since 342.3s ago] ('amazingman', 1, 0)
<User #wowow>


In [47]:
# db確認
c = User.query.filter_by(user_name="foke").first()
print(c)


2024-03-02 21:05:48,335 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2024-03-02 21:05:48,337 INFO sqlalchemy.engine.Engine [cached since 494.6s ago] ('foke', 1, 0)
None


In [52]:
# db終了
db.session.commit()
db.session.close()


### アプリケーション動作
↓ターミナル↓
```powershell
$python run_server
```


In [38]:
import requests
import json

# /v1/regist_user_data 動作確認
data = json.dumps({
    "display_name":"goodman",
    "pass_word":"goodgood"    
})

content = requests.post(
    "http://127.0.0.1:8888/v1/regist_user_data", 
    data=data, 
    headers={'Content-Type': 'application/json'}
)


In [39]:
print(content.json())

{'user_id': None}
